In [1]:
import tkinter as tk
from tkinter import filedialog, Label, Button, Frame, ttk
from PIL import Image, ImageTk
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.applications.resnet import preprocess_input
import tkinter.font as tkFont

In [2]:
# Focal Loss
def binary_focal_loss(gamma=2.0, alpha=0.25):
    import tensorflow.keras.backend as k

    def loss(y_true, y_pred):
        epsilon = k.epsilon()
        y_pred = k.clip(y_pred, epsilon, 1.0 - epsilon)
        pt_1 = tf.where(k.equal(y_true, 1), y_pred, k.ones_like(y_pred))
        pt_0 = tf.where(k.equal(y_true, 0), y_pred, k.zeros_like(y_pred))
        return -k.mean(alpha * k.pow(1.0 - pt_1, gamma) * k.log(pt_1)) - k.mean(
            (1 - alpha) * k.pow(pt_0, gamma) * k.log(1.0 - pt_0)
        )

    return loss

In [3]:
# Load model
model = load_model(
    "resnet_model_nv_mel_ES.h5", custom_objects={"loss": binary_focal_loss(2.0, 0.25)}
)

In [ ]:
THRESHOLD = 0.482
IMAGE_SIZE = (224, 224)


# Preprocess image for prediction
def preprocess_image_cv(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError(f"Could not read image at {path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMAGE_SIZE)
    return preprocess_input(img.astype("float32"))

In [9]:
# Prediction and UI update
def upload_and_predict():
    path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if not path:
        return

    # Show image
    img = Image.open(path).resize((300, 300))
    tk_img = ImageTk.PhotoImage(img)
    img_label.config(image=tk_img)
    img_label.image = tk_img

    # Predict
    x = np.expand_dims(preprocess_image_cv(path), axis=0)
    prob = model.predict(x, verbose=0)[0, 0]

    # Update progress bar
    prog_bar["value"] = prob * 100
    root.update_idletasks()

    # Set result badge
    if prob >= THRESHOLD:
        badge.config(text="❌ Malignant", bg="#e57373")
    else:
        badge.config(text="✅ Benign", bg="#81c784")
    prob_label.config(text=f"Melanoma Likelihood: {prob:.2%}")

In [23]:
# Build GUI
def build_gui():
    global root, img_label, prog_bar, badge, prob_label
    root = tk.Tk()
    root.title("Skin Lesion Melanoma Detector")
    root.configure(bg="#f7f9fc")

    # Fonts
    title_font = tkFont.Font(family="Open Sans", size=24, weight="bold")
    btn_font = tkFont.Font(family="Open Sans", size=16)

    # Title
    header = Frame(root, bg="#f7f9fc")
    header.pack(pady=20)
    Label(
        header,
        text="🔬 Skin Lesion Melanoma Detector",
        bg="#f7f9fc",
        fg="#00695c",
        font=title_font,
    ).pack()

    # Upload button
    upload = Button(
        root,
        text="📤 Upload Lesion Image",
        command=upload_and_predict,
        font=btn_font,
        bg="#4db6ac",
        fg="white",
        relief="flat",
        padx=20,
        pady=10,
    )
    upload.pack(pady=10)

    # Image display
    img_label = Label(root, bg="#f7f9fc")
    img_label.pack(pady=10)

    # Prediction frame
    pred_frame = Frame(root, bg="#f7f9fc")
    pred_frame.pack(pady=10)

    # Progress bar
    prog_bar = ttk.Progressbar(
        pred_frame, orient="horizontal", length=300, mode="determinate"
    )
    prog_bar.pack(pady=5)

    # Badge
    badge = Label(
        pred_frame,
        text="Awaiting upload...",
        font=("Open Sans", 14),
        fg="white",
        bg="#90a4ae",
        padx=10,
        pady=5,
    )
    badge.pack(pady=5)

    # Probability label
    prob_label = Label(
        pred_frame,
        text="Melanoma Likelihood: --",
        font=("Open Sans", 12),
        fg="#424242",
        bg="#f7f9fc",
    )
    prob_label.pack()

    # ABCDE panel
    abcde = Frame(root, bg="#e0f2f1", bd=1, relief="solid")
    abcde.pack(fill="x", padx=20, pady=15)
    Label(
        abcde,
        text="Signs of Melanoma:",
        font=("Open Sans", 16, "bold"),
        bg="#e0f2f1",
        fg="#004d40",
    ).pack(anchor="w", padx=10, pady=5)
    for rule in [
        "• Asymmetry",
        "• Border irregularity",
        "• Color variation",
        "• Diameter > 6mm",
        "• Evolution over time",
    ]:
        Label(
            abcde, text=rule, font=("Open Sans", 14), bg="#e0f2f1", fg="#004d40"
        ).pack(anchor="w", padx=20)

    root.mainloop()


if __name__ == "__main__":
    build_gui()